In [ ]:
cd /content/drive/MyDrive/mongodb_MapReduce/

/content/drive/MyDrive/mongodb_MapReduce


In [ ]:
import torch.tensor as Tensor
import torch as torch
import dlc_practical_prologue as prologue
import progressbar
from dlc_practical_prologue import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_input, train_target, test_input, test_target = load_data(cifar = True, one_hot_labels =True, normalize =True, flatten = True)

* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [ ]:
 train_input.shape

torch.Size([1000, 3072])

## 1.  Activation function

In [ ]:
def sigma(x):
    return torch.tanh(x)

In [ ]:
def dsigma(x):
    y = sigma(x)
    return 1 - torch.mul(y, y)

## 2. Loss

In [ ]:
def dloss(v, t):
    return 2 * (v-t)

In [ ]:
def loss(v, t):
    y = v-t
    return torch.sum(torch.mul(y, y))

## 3. Forward and backward passes

In [ ]:
def forward_pass(w1, b1, w2, b2, x):
    x0 = x
    s1 = torch.mm(w1, x0) + b1
    x1 = sigma(s1)
    s2 = torch.mm(w2, x1) + b2
    x2 = sigma(s2)
    return x0, s1, x1, s2, x2

In [ ]:
def backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2):
    dl_dx2 = dloss(x2, t)
    dl_ds2 = torch.mul(dl_dx2, dsigma(s2))
    dl_dw2 += torch.mm(dl_ds2, torch.t(x1)) 
    dl_db2 = dl_ds2 + dl_db2
    dl_dx1 = torch.mm(torch.t(w2), dl_ds2)
    dl_ds1 = torch.mul(dl_dx1, dsigma(s1))
    dl_dw1 += torch.mm(dl_ds1, torch.t(x)) 
    dl_db1 += dl_ds1 
    return dl_dx2, dl_ds2, dl_ds1, dl_dx1, dl_dw1, dl_db1, dl_dw2, dl_db2

## 4. Training the network

In [ ]:
def compute_error(test_set, test_target_set, w1, b1, w2, b2):
    n = test_set.shape[0]
    errors = 0
    for i in range(n):
        x = test_set[i].resize(test_set[i].shape[0], 1)
        t = torch.argmax(test_target_set[i])
        _, _, _, _, predicted = forward_pass(w1, b1, w2, b2, x)
        predict_class = torch.argmax(predicted, 0)
        if t != predict_class:
            nerrors += 1
    return errors/n

In [ ]:
def training():
    train, train_target, test, test_target = prologue.load_data(cifar=False, one_hot_labels=True, normalize=True, flatten = True)
    test_target = 0.9 * test_target
    train_target = 0.9 * train_target
    w1 = torch.zeros(50, 784).normal_(0, 0.000001)
    b1 = torch.zeros(50, 1).normal_(0, 0.000001)
    w2 = torch.zeros(10, 50).normal_(0, 0.000001)
    b2 = torch.zeros(10, 1).normal_(0, 0.000001)
    eta = (0.1)/(train.shape[0])
    for i in progressbar.progressbar(range(1000)):
        dl_dw1 = torch.zeros(50, 784)
        dl_db1 = torch.zeros(50, 1)
        dl_dw2 = torch.zeros(10, 50)
        dl_db2 = torch.zeros(10, 1)
        for j in range(1000):
            x = train[j].resize_(train[j].shape[0], 1)
            t = train_target[j].resize_(train_target[j].shape[0], 1)
            x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, x)
            backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2)
        w1 = w1 - eta * dl_dw1
        w2 = w2 - eta * dl_dw2
        b1 = b1 - eta * dl_db1
        b2 = b2 - eta * dl_db2
    print("The training error is: ", compute_error(train, train_target, w1, b1, w2, b2), "and the test error is: ", compute_error(test, test_target, w1, b1, w2, b2))

In [ ]:
training()

N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


100% (1000 of 1000) |####################| Elapsed Time: 0:02:07 Time:  0:02:07
/usr/local/lib/python3.7/dist-packages/torch/tensor.py:474: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


The training error is:  0.002 and the test error is:  0.153
